In [ ]:
from flask import Flask, render_template, request, redirect, url_for
from flask_sqlalchemy import SQLAlchemy
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from keras.models import Sequential, load_model
from keras.layers import Dense
from google.cloud import storage
import uuid

app = Flask(__name__)

# Replace the database URI with the appropriate URI for your global database management system
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///your_database.db'
# Ensure the following configurations are set according to your database system
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SECRET_KEY'] = 'your_secret_key_here'  # Secret key for Flask sessions

db = SQLAlchemy(app)

# User model
class User(db.Model):
    id = db.Column(db.String(36), primary_key=True, unique=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
    password = db.Column(db.String(80), nullable=False)

    def __repr__(self):
        return '<User %r>' % self.username
    
# Function to load pre-trained model
def load_pretrained_model():
    # Specify the path to the pre-trained model file
    pretrained_model_path = 'global_model_gesture_recognition.keras'

    # Load the pre-trained model
    pretrained_model = load_model(pretrained_model_path)

    return pretrained_model

# Load the pre-trained model
pretrained_model = load_pretrained_model()
    
def get_user_data_by_id(user_id):
    # Query the database for user-specific data
    motion_data = MotionData.query.filter_by(user_id=user_id).all()
    image_data = ImageData.query.filter_by(user_id=user_id).all()
    tap_data = Tap.query.filter_by(user_id=user_id).all()
    phonetic_complexity_data = phonetic_complexity.query.filter_by(user_id=user_id).all()
    phonetic_probability_data = phonetic_probability.query.filter_by(user_id=user_id).all()
    iconizity_data = iconizity.query.filter_by(user_id=user_id).all()
    gestureclass = gestureclass.query.filter_by(user_id=user_id).all()

    # Process and convert data to NumPy arrays
    motion_data = np.array([[entry.accelerometer_x, entry.accelerometer_y, entry.accelerometer_z, entry.attribute1, entry.attribute2, ...] for entry in motion_data])
    image_data = np.array([[entry.handshape, entry.thumb_position, entry.minor_location, entry.major_location, entry.attribute1, entry.attribute2, ...] for entry in image_data])
    tap_data = np.array([[entry.word, entry.sentence, entry.attribute1, entry.attribute2, ...] for entry in tap_data])
    phonetic_complexity_data = np.array([[entry.attribute1, entry.attribute2, ...] for entry in phonetic_complexity_data])
    phonetic_probability_data = np.array([[entry.attribute1, entry.attribute2, ...] for entry in phonetic_probability_data])
    iconizity_data = np.array([[entry.attribute1, entry.attribute2, ...] for entry in iconizity_data])
    gestureclass = np.array([[entry.attribute1, entry.attribute2, ...] for entry in gestureclass])

    if not gestureclass.size:
        pass

    return motion_data, image_data, tap_data, phonetic_complexity_data, phonetic_probability_data, iconizity_data

# Fine-tune the model based on user-specific data
def fine_tune_model(model, user_data):
    motion, imagesign, tap, phonetic_complexity_data, phonetic_probability_data, iconizity_data, gestureclass = user_data

    # Combine motion, imagesign, and tap data
    X = np.concatenate((motion, imagesign, tap, phonetic_complexity_data, phonetic_probability_data, iconizity_data, gestureclass), axis=1)

    # Standardize the numerical features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Predict cluster labels using the pretrained model
    gestureclass = model.predict_classes(X_scaled)

    # Define your Keras model for fine-tuning
    # Example architecture, adjust as needed
    fine_tuned_model = Sequential([
        Dense(64, activation='relu', input_shape=(X_scaled.shape[1],)),
        Dense(32, activation='relu'),
        Dense(np.max(gestureclass) + 1, activation='softmax')  # Use softmax activation for multi-class classification
    ])

    # Compile your fine-tuned model
    fine_tuned_model.compile(optimizer='adam',
                              loss='sparse_categorical_crossentropy',  # Use sparse categorical cross-entropy for integer labels
                              metrics=['accuracy'])  # Use accuracy as a metric

    # Fine-tune your model
    fine_tuned_model.fit(X_scaled, gestureclass, epochs=10, batch_size=32, validation_split=0.2)

    return fine_tuned_model

# Function to save the fine-tuned model locally
def save_model_locally(model, user_id):
    filename = f'fine_tuned_model_gesture_recognition_{user_id}.keras'
    model.save(filename)
    return filename

# Function to upload the fine-tuned model to Google Cloud Storage
def upload_model_to_cloud(filename, bucket_name, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(filename)
    print(f"Model uploaded to cloud storage: gs://{bucket_name}/{destination_blob_name}")

if __name__ == '__main__':
    app.run(debug=True)